In [1]:
from pathlib import Path
import os
from tqdm import tqdm

import torch
import torchvision
import torchinfo

!nvidia-smi

Wed Oct  9 10:44:32 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
| 29%   42C    P8             12W /  120W |    1088MiB /   6144MiB |     17%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Transfer Learning of Effective Net B2 Model will be used

In [3]:
weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
effnetb2_transforms = weights.transforms()
model = torchvision.models.efficientnet_b2(weights=weights)

1. Get Data

Model will be trained on the Food 101 Dataset (~1000 images x 101 food classes)

In [4]:
data_dir = Path().cwd().parent / "data"

# Create training transforms
train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.TrivialAugmentWide(),
        effnetb2_transforms
    ]
)

# Get the dataset (and apply the transformations)
train_dataset = torchvision.datasets.Food101(
    root=data_dir,
    split = "train",
    transform=train_transforms,
    download=True
)

test_dataset = torchvision.datasets.Food101(
    root=data_dir,
    split = "test",
    transform=effnetb2_transforms,
    download=True
)

classes_names = train_dataset.classes
num_of_classes = len(classes_names)

print(len(train_dataset), len(test_dataset))
print(classes_names)
print(num_of_classes)

75750 25250
['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons

In [5]:
# data size is too big, take percentage% of both train and test

def split_dataset(dataset, split_size=0.2, seed=42):

    length_1 = int(len(dataset) * (1-split_size))
    length_2 = len(dataset) - length_1
    
    random_split_1, random_split_2 = torch.utils.data.random_split(
        dataset, 
        lengths=[length_1, length_2],
        generator=torch.manual_seed(seed)
    )

    return random_split_1, random_split_2


# subset_percentage = 0.001

# _, train_dataset_subset = split_dataset(
#     dataset=train_dataset,
#     split_size=subset_percentage
# )

# _, test_dataset_subset = split_dataset(
#     dataset=test_dataset,
#     split_size=subset_percentage
# )
# print(len(train_dataset_subset), len(test_dataset_subset))

In [6]:
# Create a dataloader from the datasets

BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

train_dataloader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=True,
    pin_memory=True
)

test_dataloader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=False,
    pin_memory=True
)

print(len(train_dataloader), len(test_dataloader))

sample_x, sample_y = next(iter(train_dataloader))
print(sample_x)
print(sample_y)

2368 790
tensor([[[[-0.3198, -0.2171,  0.0741,  ...,  1.0502,  1.0502,  1.1529],
          [-1.1418, -0.0629,  0.3481,  ...,  1.1872,  1.1529,  1.2043],
          [-1.3130,  0.3823,  0.7933,  ...,  1.2385,  1.1700,  1.1872],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -0.9020, -0.8507, -0.9020],
          [-2.1179, -2.1179, -2.1179,  ..., -0.8849, -0.9020, -0.9363],
          [-2.1179, -2.1179, -2.1179,  ..., -0.8849, -0.8849, -0.9020]],

         [[ 0.2227,  0.4153,  0.8880,  ..., -0.1275,  0.0476,  0.2752],
          [-0.7927,  0.5903,  1.1155,  ...,  0.0126,  0.1176,  0.3102],
          [-1.0203,  1.1331,  1.5882,  ...,  0.1352,  0.2052,  0.3627],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -0.7052, -0.6702, -0.7052],
          [-2.0357, -2.0357, -2.0357,  ..., -0.7227, -0.7227, -0.7577],
          [-2.0357, -2.0357, -2.0357,  ..., -0.7577, -0.7577, -0.7752]],

         [[ 1.2108,  1.3502,  1.7337,  ..., -0.3927, -0.0964,  0.1999],
          [-0.1487,  

2. Base model creation

In [7]:
weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
effnetb2_transforms = weights.transforms()
model = torchvision.models.efficientnet_b2(weights=weights)

In [8]:
# freeze the parameters
for param in model.parameters():
    param.requires_grad = False

# change the output layer
# print(model.classifier) # to modify only the out_features = num of classes
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.3, inplace=True),
    torch.nn.Linear(
        in_features=1408,
        out_features=num_of_classes,
        bias=True
    )
)

In [9]:
model = model.to(device=device)

3. Training

In [10]:
# EPOCHS = 100



# for epoch in tqdm(range(EPOCHS)):

#     training_loss = 0
#     training_acc = 0
#     testing_loss = 0
#     testing_acc = 0

#     # Train step
#     model.train()

#     for batch_no, (X_train, y_train) in enumerate(train_dataloader):

#         X_train, y_train = X_train.to(device), y_train.to(device)

#         y_logits = model(X_train)

#         loss = loss_fn(y_logits, y_train)
#         training_loss += loss

#         y_pred_classes = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)
#         training_acc += (y_pred_classes == y_train).sum().item() / len(y_train)

#         optimizer.zero_grad()

#         loss.backward()

#         optimizer.step()

#         if batch_no % (len(train_dataloader)//20) == 0:
#             print(f"Training batch {batch_no} / {len(train_dataloader)}")

#     # Evaluation step
#     model.eval()

#     with torch.no_grad():

#         for X_test, y_test in test_dataloader:

#             X_test, y_test = X_test.to(device), y_test.to(device)

#             y_logits = model(X_test)

#             loss = loss_fn(y_logits, y_test)
#             testing_loss += loss

#             y_pred_classes = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)
#             testing_acc += (y_pred_classes == y_test).sum().item() / len(y_test)
    
#     results["training_loss"] = training_loss/len(train_dataloader)
#     results["training_acc"] = training_acc/len(train_dataloader)
#     results["testing_loss"] = testing_loss/len(test_dataloader)
#     results["testing_acc"] = testing_acc/len(test_dataloader)

#     print(f"Epoch {epoch}")
#     print(f"Training Loss = {results["training_loss"]}  ||  Training Accuracy = {results["training_acc"]}")
#     print(f"Testing Loss = {results["testing_loss"]}  ||  Testing Accuracy = {results["testing_acc"]}")

In [11]:
# Create loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=0.001
)

In [12]:
def train_step(model, train_dataloader, loss_fn, optimizer, device):

    training_loss = 0
    training_acc = 0

    # Train step
    model.train()

    for batch_no, (X_train, y_train) in enumerate(train_dataloader):

        X_train, y_train = X_train.to(device), y_train.to(device)

        y_logits = model(X_train)

        loss = loss_fn(y_logits, y_train)
        training_loss += loss

        y_pred_classes = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)
        training_acc += (y_pred_classes == y_train).sum().item() / len(y_train)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        # show rough progress
        if batch_no % (len(train_dataloader)//20) == 0:
            print(f"Training batch {batch_no} / {len(train_dataloader)}")

    return training_loss/len(train_dataloader), training_acc/len(train_dataloader)


def eval_step(model, test_dataloader, loss_fn, device):

    test_loss = 0
    test_acc = 0

    # Evaluation step
    model.eval()

    with torch.no_grad():

        for X_test, y_test in test_dataloader:

            X_test, y_test = X_test.to(device), y_test.to(device)

            y_logits = model(X_test)

            loss = loss_fn(y_logits, y_test)
            test_loss += loss

            y_pred_classes = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)
            test_acc += (y_pred_classes == y_test).sum().item() / len(y_test)
    
    return test_loss/len(test_dataloader), test_acc/len(test_dataloader)


def train(model, train_dataloader, test_dataloader, loss_fn, optimizer, epochs = 10, device="cpu"):

    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    for epoch in tqdm(range(epochs)):
        
        train_loss, train_acc = train_step(
            model=model,
            train_dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device
        )

        ### Commented out eval portions on test set for quicker training
        # test_loss, test_acc = eval_step(
        #     model=model,
        #     test_dataloader=test_dataloader,
        #     loss_fn=loss_fn,
        #     device=device
        # )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        # results["test_loss"].append(test_loss)
        # results["test_acc"].append(test_acc)

        print(f"Epoch {epoch}")
        print(f"Train Loss = {train_loss}  ||  Training Accuracy = {train_acc}")
        # print(f"Test Loss = {test_loss}  ||  Testing Accuracy = {test_acc}")

    return results

In [13]:
EPOCHS = 100

results = train(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=EPOCHS,
    device=device
)

  0%|          | 0/100 [00:00<?, ?it/s]

Training batch 0 / 2368
Training batch 118 / 2368
Training batch 236 / 2368
Training batch 354 / 2368
Training batch 472 / 2368
Training batch 590 / 2368
Training batch 708 / 2368
Training batch 826 / 2368
Training batch 944 / 2368
Training batch 1062 / 2368
Training batch 1180 / 2368
Training batch 1298 / 2368
Training batch 1416 / 2368
Training batch 1534 / 2368
Training batch 1652 / 2368
Training batch 1770 / 2368
Training batch 1888 / 2368
Training batch 2006 / 2368
Training batch 2124 / 2368
Training batch 2242 / 2368
Training batch 2360 / 2368


  1%|          | 1/100 [19:34<32:17:06, 1174.01s/it]

Epoch 0
Train Loss = 2.987398147583008  ||  Training Accuracy = 0.40506228885135137
Training batch 0 / 2368
Training batch 118 / 2368
Training batch 236 / 2368
Training batch 354 / 2368
Training batch 472 / 2368
Training batch 590 / 2368
Training batch 708 / 2368
Training batch 826 / 2368
Training batch 944 / 2368
Training batch 1062 / 2368
Training batch 1180 / 2368
Training batch 1298 / 2368
Training batch 1416 / 2368
Training batch 1534 / 2368
Training batch 1652 / 2368
Training batch 1770 / 2368
Training batch 1888 / 2368
Training batch 2006 / 2368
Training batch 2124 / 2368
Training batch 2242 / 2368
Training batch 2360 / 2368


  2%|▏         | 2/100 [27:03<20:21:16, 747.72s/it] 

...

Epoch 97
Train Loss = 2.594062328338623  ||  Training Accuracy = 0.5001803561373873
Training batch 0 / 2368
Training batch 118 / 2368
Training batch 236 / 2368
Training batch 354 / 2368
Training batch 472 / 2368
Training batch 590 / 2368
Training batch 708 / 2368
Training batch 826 / 2368
Training batch 944 / 2368
Training batch 1062 / 2368
Training batch 1180 / 2368
Training batch 1298 / 2368
Training batch 1416 / 2368
Training batch 1534 / 2368
Training batch 1652 / 2368
Training batch 1770 / 2368
Training batch 1888 / 2368
Training batch 2006 / 2368
Training batch 2124 / 2368
Training batch 2242 / 2368
Training batch 2360 / 2368


 99%|█████████▉| 99/100 [10:31:54<07:28, 448.26s/it]

Epoch 98
Train Loss = 2.5935919284820557  ||  Training Accuracy = 0.5008665892454954
Training batch 0 / 2368
Training batch 118 / 2368
Training batch 236 / 2368
Training batch 354 / 2368
Training batch 472 / 2368
Training batch 590 / 2368
Training batch 708 / 2368
Training batch 826 / 2368
Training batch 944 / 2368
Training batch 1062 / 2368
Training batch 1180 / 2368
Training batch 1298 / 2368
Training batch 1416 / 2368
Training batch 1534 / 2368
Training batch 1652 / 2368
Training batch 1770 / 2368
Training batch 1888 / 2368
Training batch 2006 / 2368
Training batch 2124 / 2368
Training batch 2242 / 2368
Training batch 2360 / 2368


100%|██████████| 100/100 [10:41:23<00:00, 384.83s/it]

Epoch 99
Train Loss = 2.588791608810425  ||  Training Accuracy = 0.5010205518018018


In [14]:
model_path = Path().cwd().parent / "models" / "pretrained_effnetb2_food101.pth"
torch.save(model.state_dict(), 'model_weights.pth')